In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"


import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers
from keras.layers import StringLookup
import math

In [2]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe' # or 'notebook' or 'colab' or 'jupyterlab'

In [3]:
from mlp import Params

# Data
- data set will be used as sample data which is from kaggle dataset.

In [4]:
class Data:
    data_path = "/Volumes/PS2000W/instacart-market-basket-analysis/"
    order_products__prior = pd.read_csv(
        data_path + "order_products__prior.csv")
    order_products__train = pd.read_csv(
        data_path + "order_products__train.csv")
    orders = pd.read_csv(data_path + "orders.csv")
    products = pd.read_csv(data_path + "products.csv")
    

In [5]:
# this update is needed when tensorflow detects numbers as intetegers even it is conterted to strings
def convert_to_str(x):
    return 'b_' + str(int(x))

In [6]:
Data.products['product_id'] = Data.products['product_id'].apply(convert_to_str)
Data.products['aisle_id'] = Data.products['aisle_id'].apply(convert_to_str)
Data.products['department_id'] = Data.products['department_id'].apply(convert_to_str)
Data.orders['user_id'] = Data.orders['user_id'].apply(convert_to_str)
Data.order_products__train['product_id'] = Data.order_products__train['product_id'].apply(convert_to_str)
Data.order_products__prior['product_id'] = Data.order_products__prior['product_id'].apply(convert_to_str)

In [7]:
Data.train = (
    Data.order_products__train
    .merge(
        Data.products, 
        on='product_id', 
        how='left'
    ).merge(
        Data.orders
        .query("eval_set == 'train'")
        [[
            'order_id', 
            'user_id', 
            'order_dow', 
            'order_hour_of_day', 
            'order_number'
        ]],
        on='order_id',
        how='left'
    )
)

In [8]:
Data.prior = (
    Data.order_products__prior
    .merge(
        Data.products, 
        on='product_id', 
        how='left'
    ).merge(
        Data.orders
        .query("eval_set == 'prior'")
        [[
            'order_id', 
            'user_id', 
            'order_dow', 
            'order_hour_of_day', 
            'order_number'
        ]],
        on='order_id',
        how='left'
    )
)

In [9]:
Data.train['ts'] =  (
    Data.train
    .sort_values(["user_id", "order_number", "add_to_cart_order"])
    .groupby("user_id")
    .cumcount() + 1
) 

In [10]:
Data.train.head(1).T

,0
order_id,1
product_id,b_49302
add_to_cart_order,1
reordered,1
product_name,Bulgarian Yogurt
aisle_id,b_120
department_id,b_16
user_id,b_112108
order_dow,4
order_hour_of_day,10


In [11]:
Data.users = pd.DataFrame(
    Data.train.user_id.unique().tolist(),
    columns=['user_id']
)

# EDA

### product order cnt & product user cnt

In [12]:
product_order_cnt = (
    Data.order_products__train
    .groupby("product_id")
    .order_id
    .count()
    .reset_index()
    .rename(columns={"order_id": "p_order_cnt"})
    .sort_values('p_order_cnt', ascending=False)
)

_min = product_order_cnt.p_order_cnt.min() 
_max = product_order_cnt.p_order_cnt.max()

product_order_cnt['p_order_cnt_norm'] = (
    (product_order_cnt['p_order_cnt'] - _min)
    / (_max - _min)
)
product_order_cnt = product_order_cnt.fillna(0)

In [13]:
Data.order_products__train.merge(
    Data.orders[['order_id', 'user_id']],
    on='order_id',
    how='left'
)

,order_id,product_id,add_to_cart_order,reordered,user_id
0,1,b_49302,1,1,b_112108
1,1,b_11109,2,1,b_112108
2,1,b_10246,3,0,b_112108
3,1,b_49683,4,0,b_112108
4,1,b_43633,5,1,b_112108
...,...,...,...,...,...
1384612,3421063,b_14233,3,1,b_169679
1384613,3421063,b_35548,4,1,b_169679
1384614,3421070,b_35951,1,1,b_139822
1384615,3421070,b_16953,2,1,b_139822


In [14]:
product_user_cnt = (
    Data.order_products__train.merge(
        Data.orders[['order_id', 'user_id']],
        on='order_id',
        how='left'
    )
    .groupby("product_id")
    .user_id
    .agg(pd.Series.nunique)
    .reset_index()
    .rename(columns={"user_id": "p_user_cnt"})
    .sort_values('p_user_cnt', ascending=False)
)

_min = product_user_cnt.p_user_cnt.min() 
_max = product_user_cnt.p_user_cnt.max()

product_user_cnt['p_user_cnt_norm'] = (
    (product_user_cnt['p_user_cnt'] - _min)
    / (_max - _min)
)
product_user_cnt = product_user_cnt.fillna(0)

In [15]:
Data.products = (
    Data.products
    .merge(
        product_user_cnt[['product_id', 'p_user_cnt']],
        on='product_id',
        how='left'
    ).merge(
        product_order_cnt[['product_id', 'p_order_cnt']],
        on='product_id',
        how='left'
    )
)
Data.products.head(1).T

,0
product_id,b_1
product_name,Chocolate Sandwich Cookies
aisle_id,b_61
department_id,b_19
p_user_cnt,76.0
p_order_cnt,76.0


### user order cnt

In [16]:
user_order_cnt = (
    Data.orders
    .groupby("user_id")
    .order_id
    .agg(pd.Series.nunique)
    .reset_index()
    .rename(columns={"order_id": "u_order_cnt"})
)
fig = px.histogram(user_order_cnt, x="u_order_cnt", nbins=20)
fig.show()

In [17]:
Data.users  = Data.users.merge(
    user_order_cnt, 
    on='user_id', 
    how='left'
)
Data.users.head(1).T

,0
user_id,b_112108
u_order_cnt,4


In [18]:
### user product cnt

In [19]:
user_product_cnt = (
    Data.train
    .groupby("user_id")
    .product_id
    .agg(pd.Series.nunique)
    .reset_index()
    .rename(columns={"product_id": "u_product_cnt"})
)
fig = px.histogram(user_product_cnt, x="u_product_cnt", nbins=20)
fig.show()

In [20]:
Data.users  = Data.users.merge(
    user_product_cnt, 
    on='user_id', 
    how='left'
)
Data.users.head(1).T

,0
user_id,b_112108
u_order_cnt,4
u_product_cnt,8


### rankings

In [21]:
rankings = (
    Data.order_products__train
    [['order_id', 'product_id', 'add_to_cart_order', 'reordered']]
    .merge(
        product_order_cnt[['product_id', 'p_order_cnt_norm']],
        on='product_id',
        how='left'
    )
    .merge(
        product_user_cnt[['product_id', 'p_user_cnt_norm']],
        on='product_id',
        how='left'
    )
)
rankings.head()

,order_id,product_id,add_to_cart_order,reordered,p_order_cnt_norm,p_user_cnt_norm
0,1,b_49302,1,1,0.000374,0.000374
1,1,b_11109,2,1,0.007637,0.007637
2,1,b_10246,3,0,0.056662,0.056662
3,1,b_49683,4,0,0.128812,0.128812
4,1,b_43633,5,1,0.001228,0.001228


In [22]:
rankings['relevance_scores'] = (
    (.1 * rankings['add_to_cart_order'])
    + (.3 * rankings['reordered'])
    + (.3 * rankings['p_order_cnt_norm'])
    + (.3 * rankings['p_user_cnt_norm'])
)

In [23]:
fig = px.histogram(rankings, x="relevance_scores", nbins=20)
fig.show()

In [24]:
def get_ranking(r):
    if r <= .5:
        return 1
    if .5 < r <= 1.5:
        return 2
    if 1.5 < r <= 2:
        return 3
    if 2 < r <= 3.5:
        return 4
    if 3.5 < r:
        return 5

rankings['rating'] = rankings.relevance_scores.apply(
    get_ranking
)
rankings

,order_id,product_id,add_to_cart_order,reordered,p_order_cnt_norm,p_user_cnt_norm,relevance_scores,rating
0,1,b_49302,1,1,0.000374,0.000374,0.400224,1
1,1,b_11109,2,1,0.007637,0.007637,0.504582,2
2,1,b_10246,3,0,0.056662,0.056662,0.333997,1
3,1,b_49683,4,0,0.128812,0.128812,0.477287,1
4,1,b_43633,5,1,0.001228,0.001228,0.800737,2
...,...,...,...,...,...,...,...,...
1384612,3421063,b_14233,3,1,0.029533,0.029533,0.617720,2
1384613,3421063,b_35548,4,1,0.000908,0.000908,0.700545,2
1384614,3421070,b_35951,1,1,0.116155,0.116155,0.469693,1
1384615,3421070,b_16953,2,1,0.052870,0.052870,0.531722,2


In [25]:
rankings.groupby("rating").product_id.agg(pd.Series.nunique)

rating
1    25904
2    33166
3    20278
4    19777
5     7342
Name: product_id, dtype: int64

In [26]:
Data.train = (
    Data.train
    .merge(
        rankings[['order_id', 'product_id', 'rating']], 
        on=['order_id', 'product_id'], how='left'
    )
)

In [27]:
Data.train.head(1).T

,0
order_id,1
product_id,b_49302
add_to_cart_order,1
reordered,1
product_name,Bulgarian Yogurt
aisle_id,b_120
department_id,b_16
user_id,b_112108
order_dow,4
order_hour_of_day,10


# pre-process

In [30]:
lookups = [
    "user_id",
    "product_id",
    "order_dow",
    "order_number",
    "order_hour_of_day",
    "aisle_id",
    "department_id"
]


sequential_features = [
    "product_id",
    "rating", 
]


user_features = [
    "user_order_cnt",
    "order_hour_of_day",
    
]


item_features = [
    "p_order_cnt",
    "p_user_cnt",
    "department_id",
    "aisle_id",
    
    
]

sequence_length = 4 
step_size = 2


params = {
    "user_id": "user_id",
    "item_id": "product_id",
    "sequence_length": 4,
    "num_heads": 3,
    "hidden_layers": 2,
    "hidden_units": 256,
    "dropout_rate": 0.1
}

categorical_features = [
    'user_id', 
    'sequence_product_ids', 
    'sequence_ratings',
    'target_product_id',
    'aisle_id',
    'department_id'
]
numeric_features = [
    'p_user_cnt', 
    'p_order_cnt', 
    'u_order_cnt',
    'u_product_cnt'
]
target = "target"


lookup_features = list(
    set(categorical_features) - set(['sequence_ratings'])
)

In [31]:
for cats in lookups:
    Data.train[cats] = Data.train[cats].astype(str)

In [32]:
train_data = (
    Data.train
    .sort_values(["user_id", "ts"])
    .groupby("user_id") 
)

In [33]:
train_data = pd.DataFrame(
    {
        "user_id": list(train_data.groups.keys()),
        "product_ids": list(train_data['product_id'].apply(list)),
        "ratings": list(train_data['rating'].apply(list))
    }
)

In [34]:
def convert_to_str(seq):
    return ",".join([str(s) for s in seq])

    
def create_sequences(values, window_size, step_size):    
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) < window_size:
                seq = seq + ([seq[-1]] * (window_size - len(seq)))
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences

In [35]:
for seq in ['product_ids', 'ratings']:
    train_data["sequence_"+seq] = train_data[seq].apply(
        lambda row: 
        create_sequences(
            row, 
            sequence_length,
            step_size
        )
    )

In [36]:
train_data.head(1).T

,0
user_id,b_1
product_ids,"[b_196, b_25133, b_38928, b_26405, b_39657, b_..."
ratings,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
sequence_product_ids,"[[b_196, b_25133, b_38928, b_26405], [b_38928,..."
sequence_ratings,"[[1, 2, 2, 2], [2, 2, 2, 2], [2, 2, 2, 2], [2,..."


In [37]:
train_data = (
    train_data
    [['user_id', 'sequence_product_ids', 'sequence_ratings']]
    .explode(
        ['sequence_product_ids', 'sequence_ratings'], 
    )
)

train_data.head(2)

,user_id,sequence_product_ids,sequence_ratings
0,b_1,"[b_196, b_25133, b_38928, b_26405]","[1, 2, 2, 2]"
0,b_1,"[b_38928, b_26405, b_39657, b_10258]","[2, 2, 2, 2]"


In [38]:
train_data[[
    'sequence_product_ids',
    'sequence_ratings',
    'target_product_id', 
    'target'
]] = train_data.apply(
    lambda row:
    pd.Series([
        row['sequence_product_ids'][:-1],
        row['sequence_ratings'][:-1],
        row['sequence_product_ids'][-1], 
        row['sequence_ratings'][-1]
    ]),
    axis=1
)

In [39]:
train_data.head(1).T

,0
user_id,b_1
sequence_product_ids,"[b_196, b_25133, b_38928]"
sequence_ratings,"[1, 2, 2]"
target_product_id,b_26405
target,2


In [40]:
train_data = (
    train_data.merge(
        (
            Data.products
            .rename(columns={"product_id": "target_product_id"})
        ), 
        on='target_product_id',
        how='left'
    ).merge(
        Data.users,
        on='user_id',
        how='left'
    )
)
train_data.head(1).T

,0
user_id,b_1
sequence_product_ids,"[b_196, b_25133, b_38928]"
sequence_ratings,"[1, 2, 2]"
target_product_id,b_26405
target,2
product_name,XL Pick-A-Size Paper Towel Rolls
aisle_id,b_54
department_id,b_17
p_user_cnt,105.0
p_order_cnt,105.0


# feature_selection

In [41]:
categorical_features = [
    'user_id', 
    'sequence_product_ids', 
    'sequence_ratings',
    'target_product_id',
    'aisle_id',
    'department_id'
]
numeric_features = [
    'p_user_cnt', 
    'p_order_cnt', 
    'u_order_cnt',
    'u_product_cnt'
]
target = "target"

In [42]:
train_data = train_data[
    categorical_features 
    + numeric_features
    + [target]
]

In [43]:
for num in numeric_features+['target']:
    train_data[num] = train_data[num].astype(float)

In [44]:
del product_order_cnt, product_user_cnt

In [45]:
del user_order_cnt, user_product_cnt

# train - validation - test split

In [46]:
split_ratio = 0.85

In [47]:
random_selection = np.random.rand(len(train_data.index)) <= split_ratio
val_dataset = train_data[~random_selection]
train_dataset = train_data[random_selection]

In [48]:
val_dataset.shape, train_dataset.shape

((83829, 11), (472611, 11))

# Lookups & Encoders

In [49]:
class Encoder:
    def __init__(self, params, lookup_features):
        self.params = params
        self.item_id = params.get('item_id')
        self.target_item_id = f"target_{params.get('item_id')}"
        self.sequence_item_ids = 'sequence_' + params.get('item_id') + 's'
        self.sequence_length = params.get('sequence_length')
        self.positions = tf.range(start=0, limit=self.sequence_length - 1, delta=1)
        self.lookups = {}
        self.lookup_features = lookup_features
        self.item_lookup_features = [self.target_item_id, self.sequence_item_ids]
        self.embedding_encoders = {}
        self.embedding_dims = {}
        self.item_embedding_processor = None
        self.position_embedding_encoder = None

    @classmethod
    def generate(cls, train_data, params, products, lookup_features):
        _cls = Encoder(
            params=params,
            lookup_features=lookup_features
        )
        _cls.get_lookups(
            train_data,
            products
        )
        return _cls

    def update_lookups_and_embeddings(self, vocabulary, lookup):
        self.lookups[lookup] = StringLookup(
            vocabulary=vocabulary, mask_token=None, oov_token=0,  num_oov_indices=1)
        self.embedding_dims[lookup] = int(math.sqrt(len(vocabulary)))
        self.embedding_encoders[lookup] = layers.Embedding(
                input_dim=len(vocabulary)+1,
                output_dim=self.embedding_dims[lookup],
                name=f"{lookup}_embedding",
            )

    def get_lookups(self, train_data, products):
        for lookup in self.lookup_features:
            if lookup not in self.item_lookup_features: 
                # Convert the string input values into integer indices.
                vocabulary = train_data[lookup].astype(str).unique().tolist()
                self.update_lookups_and_embeddings(vocabulary, lookup)
                                              
        # item Id embedding and lookups
        vocabulary = products[self.item_id].astype(str).unique().tolist()
        self.update_lookups_and_embeddings(vocabulary, self.item_id)
        self.item_embedding_processor = layers.Dense(
            units=self.embedding_dims[self.item_id],
            activation="relu",
            name=f"process_{self.item_id}_embedding",
        )
        self.position_embedding_encoder = layers.Embedding(
            input_dim=self.sequence_length - 1,
            output_dim=self.embedding_dims[self.item_id],
            name="position_embedding",
        )

    def query(self, inp, lookup):
        return self.embedding_encoders[lookup](inp)

    def item_embeddings(self, inputs):
        emb_target = self.query(inputs[self.target_item_id], self.item_id)
        emb_target = self.item_embedding_processor(emb_target)
        emb_seq = self.query(inputs[self.sequence_item_ids], self.item_id)
        emb_seq = self.item_embedding_processor(emb_seq)
        return emb_target, emb_seq

    def get_embeddings(self, inputs):
        encoded = []
        encoded_transformer = []
        for lookup in self.lookup_features:
            if lookup not in self.item_lookup_features: 
                print(inputs[lookup])
                encoded.append(self.query(inputs[lookup], lookup))
        
        ## Create a single embedding vector for the user features
        if len(encoded) > 1:
            encoded = layers.concatenate(encoded)
        elif len(encoded) == 1:
            encoded = encoded[0]
        else:
            encoded = None

        (
            encoded_target_item, 
            encoded_sequence_items
        ) = self.item_embeddings(
            inputs
        )        
        encodded_positions = self.position_embedding_encoder(self.positions)
        sequence_ratings = keras.ops.expand_dims(inputs["sequence_ratings"], -1)

        encoded_sequence_items_with_poistion_and_rating = layers.Multiply()(
            [(encoded_sequence_items + encodded_positions), sequence_ratings]
        )

        # Construct the transformer inputs.
        for i in range(self.sequence_length - 1):
            feature = encoded_sequence_items_with_poistion_and_rating[:, i, ...]
            feature = keras.ops.expand_dims(feature, 1)
            encoded_transformer.append(feature)

        encoded_transformer = layers.concatenate(
            encoded_transformer, axis=1
        )
            
        return encoded_transformer, encoded            
        

In [50]:
encoders = Encoder.generate(
    train_data=train_data,
    params=params,
    products=Data.products,
    lookup_features=lookup_features
)

In [51]:
encoders.lookups

{'department_id': <StringLookup name=string_lookup, built=False>,
 'aisle_id': <StringLookup name=string_lookup_1, built=False>,
 'user_id': <StringLookup name=string_lookup_2, built=False>,
 'product_id': <StringLookup name=string_lookup_3, built=False>}

# create `tf.data.Dataset`

In [52]:
train_feature_dataset = {
    encoders.target_item_id: tf.reshape(
        encoders.lookups[encoders.item_id](train_dataset[encoders.target_item_id]), (len(train_dataset),1)
    ),
    encoders.sequence_item_ids: (
        encoders.lookups[encoders.item_id](train_dataset[encoders.sequence_item_ids].tolist())),
    "sequence_ratings": tf.cast(train_dataset['sequence_ratings'].tolist(), tf.float32),
    "target": tf.reshape(tf.cast(train_dataset['target'], tf.float32), (len(train_dataset),1))
}

In [53]:
for f in lookup_features:
    if f not in train_feature_dataset:
        train_feature_dataset[f] = tf.reshape(
            encoders.lookups[f](train_dataset[f]), (len(train_dataset),1))

In [54]:
for f in numeric_features:
    train_feature_dataset[f] = tf.reshape(tf.cast(train_dataset[f], tf.float32), (len(train_dataset), 1))

In [55]:
def preprocess(feature):
    target = feature['target']
    del feature['target']
    return feature, target

In [56]:
train_feature_dataset = tf.data.Dataset.from_tensor_slices(train_feature_dataset)
train_feature_dataset = train_feature_dataset.cache()
train_feature_dataset = train_feature_dataset.shuffle(5000)
train_feature_dataset = train_feature_dataset.prefetch(tf.data.AUTOTUNE)
train_feature_dataset = train_feature_dataset.batch(32)

In [57]:
train_feature_dataset = train_feature_dataset.map(preprocess)

In [58]:
for i in train_feature_dataset.take(1):
    print()
i[0]

2025-01-05 20:45:25.379481: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-01-05 20:45:25.385809: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


{'target_product_id': <tf.Tensor: shape=(32, 1), dtype=int64, numpy=
 array([[18465],
        [  493],
        [28204],
        [42526],
        [24410],
        [25947],
        [ 5869],
        [31683],
        [17429],
        [33651],
        [47194],
        [24114],
        [24838],
        [17518],
        [12341],
        [21137],
        [27104],
        [17600],
        [26620],
        [29801],
        [23181],
        [45007],
        [25069],
        [33754],
        [45948],
        [39877],
        [ 1643],
        [43352],
        [48925],
        [ 9517],
        [13176],
        [ 2131]])>,
 'sequence_product_ids': <tf.Tensor: shape=(32, 3), dtype=int64, numpy=
 array([[ 4756, 25832, 18465],
        [27966, 13176,  5550],
        [48220, 28985, 31635],
        [18288,  3211, 25636],
        [43072, 14156, 29061],
        [ 5876, 28985, 34881],
        [14766, 22935, 24964],
        [24852, 19348, 16213],
        [ 5450, 27336, 26497],
        [47993, 39127, 42697],
  

In [59]:
i[1]

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [1.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [4.],
       [2.],
       [3.],
       [3.],
       [2.],
       [2.],
       [5.],
       [3.],
       [2.],
       [2.],
       [2.],
       [2.],
       [3.],
       [2.],
       [3.],
       [2.]], dtype=float32)>

In [60]:
class Inputs:
    def __init__(
        self, 
        params,
        categorical_features, 
        numeric_features
    ):
        self.params = params
        self.item_id = params.get('item_id')
        self.target_item_id = f"target_{params.get('item_id')}"
        self.sequence_item_ids = 'sequence_' + params.get('item_id') + 's'
        self.sequence_length = params.get('sequence_length')
        self.categorical_features = categorical_features
        self.numeric_features = numeric_features
        self.inputs = {}
        self.collect_inputs()

    def collect_inputs(self):
        for cat in self.categorical_features:
            if cat == self.sequence_item_ids:
                self.inputs[cat] = keras.Input(
                    name=cat, shape=(self.sequence_length - 1,)
                )
            elif cat == "sequence_ratings":
                self.inputs[cat] = keras.Input(
                    name="sequence_ratings", shape=(self.sequence_length - 1,)
                )
            else:
                self.inputs[cat] = keras.Input(name=cat, shape=(1,))

        for num in self.numeric_features:
            self.inputs[num] = keras.Input(name=num, shape=(1,)) 

In [61]:
inputs = Inputs(
        params,
        categorical_features, 
        numeric_features
)

In [62]:
inputs.inputs.keys()

dict_keys(['user_id', 'sequence_product_ids', 'sequence_ratings', 'target_product_id', 'aisle_id', 'department_id', 'p_user_cnt', 'p_order_cnt', 'u_order_cnt', 'u_product_cnt'])

In [63]:
inputs.inputs

{'user_id': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=user_id>,
 'sequence_product_ids': <KerasTensor shape=(None, 3), dtype=float32, sparse=False, name=sequence_product_ids>,
 'sequence_ratings': <KerasTensor shape=(None, 3), dtype=float32, sparse=False, name=sequence_ratings>,
 'target_product_id': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=target_product_id>,
 'aisle_id': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=aisle_id>,
 'department_id': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=department_id>,
 'p_user_cnt': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=p_user_cnt>,
 'p_order_cnt': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=p_order_cnt>,
 'u_order_cnt': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=u_order_cnt>,
 'u_product_cnt': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=u_product_cnt>}

In [64]:
class Transformer:
    def __init__(self, params, inputs: Inputs, encoders: Encoder):
        self.params = params
        self.num_heads = params.get('num_heads')
        self.dropout_rate = params.get('dropout_rate')
        self.inputs = inputs
        self.encoders = encoders
        self.hidden_units = self.cal_hidden_layer_of_units(
            params.get('hidden_layers'),
            params.get('hidden_units')
        )

    @staticmethod
    def cal_hidden_layer_of_units(
        hidden_layers, _encoding_dim, autoencoder_layers=False
    ):
        """creating hidden layers for each tower
        hidden_layers:
            number of hidden layer that will be created
        _encoding_dim:
            number of hidden unit that will be used in first hidden layer
        autoencoder_layers:
            if it is for autoencoder, process will not be same. hidden unit will be decreasing for each hidden layer,
            however, for autoencoder, after seeing bottle_neck unit, unit size will be re-increasing till _encoding_dim
        how it works;
            1st example_configurations;
                hidden_layers      : 3
                _encoding_dim      : 16
                autoencoder_layers : False
                layers             : 16 - 8 (16/2) - 4 (8/2) - 2 (4/2)
            2nd example_configurations;
                hidden_layers      : 3
                _encoding_dim      : 16
                autoencoder_layers : True
                layers             : 16 - 8 (16/2) - 4 (8/2) - 2 (4/2) (bottle_neck) - 4 (2*2) - 8 (4*2) - 16 (8*2)
        """
        count = 1
        _unit = _encoding_dim
        h_l_units = []
        while count != hidden_layers + 1:
            h_l_units.append(int(_unit))
            _unit /= 2
            if int(_unit) == 1:
                count = hidden_layers + 1
            else:
                count += 1
        if autoencoder_layers:
            count = 1
            while count != hidden_layers + 2:
                h_l_units.append(int(_unit))
                _unit *= 2
                count += 1
        return h_l_units

    def create_model(self):
        transformer_features, other_features = self.encoders.get_embeddings(
            self.inputs.inputs
        )
        attention_output = layers.MultiHeadAttention(
            num_heads=self.num_heads, 
            key_dim=transformer_features.shape[2], 
            dropout=self.dropout_rate
        )(transformer_features, transformer_features)
    
        # Transformer block.
        attention_output = layers.Dropout(self.dropout_rate)(attention_output)
        x1 = layers.Add()([transformer_features, attention_output])
        x1 = layers.LayerNormalization()(x1)
        x2 = layers.LeakyReLU()(x1)
        x2 = layers.Dense(units=x2.shape[-1])(x2)
        x2 = layers.Dropout(self.dropout_rate)(x2)
        transformer_features = layers.Add()([x1, x2])
        transformer_features = layers.LayerNormalization()(transformer_features)
        features = layers.Flatten()(transformer_features)
    
        # Included the other features.
        if other_features is not None:
            features = layers.concatenate(
                [features, layers.Reshape([other_features.shape[-1]])(other_features)]
            )
    
        # Fully-connected layers.
        for num_units in self.hidden_units:
            features = layers.Dense(num_units)(features)
            features = layers.BatchNormalization()(features)
            features = layers.LeakyReLU()(features)
            features = layers.Dropout(self.dropout_rate)(features)
    
        outputs = layers.Dense(units=1)(features)
        model = keras.Model(inputs=self.inputs.inputs, outputs=outputs)
        return model
        

In [65]:
transformer = Transformer(
    params,
    inputs,
    encoders
)

In [67]:
model = transformer.create_model()

<KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=department_id>
<KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=aisle_id>
<KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=user_id>


In [68]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sequence_product_i… │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ product_id_embeddi… │ (None, 3, 222)    │ 11,030,958 │ sequence_product… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ process_product_id… │ (None, 3, 222)    │     49,506 │ product_id_embed… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequence_ratings    │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 3, 222)    │          0 │ process_product_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expand_dims         │ (None, 3, 1)      │          0 │ sequence_ratings… │
│ (ExpandDims)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 3, 222)    │          0 │ add[0][0],        │
│ (Multiply)          │                   │            │ expand_dims[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 222)       │          0 │ multiply_1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 222)       │          0 │ multiply_1[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 222)       │          0 │ multiply_1[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expand_dims_1       │ (None, 1, 222)    │          0 │ get_item[0][0]    │
│ (ExpandDims)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expand_dims_2       │ (None, 1, 222)    │          0 │ get_item_1[0][0]  │
│ (ExpandDims)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expand_dims_3       │ (None, 1, 222)    │          0 │ get_item_2[0][0]  │
│ (ExpandDims)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 3, 222)    │          0 │ expand_dims_1[0]… │
│ (Concatenate)       │                   │            │ expand_dims_2[0]… │
│                     │                   │            │ expand_dims_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 3, 222)    │    593,628 │ concatenate_3[0]… │
│ (MultiHeadAttentio… │                   │            │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 3, 222)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 3, 222)    │          0 │ concatenate_3[0]… │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 59,525,904 (227.07 MB)

 Trainable params: 59,525,136 (227.07 MB)

 Non-trainable params: 768 (3.00 KB)

In [69]:
model.compile(
    optimizer=keras.optimizers.Adagrad(learning_rate=0.01),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanAbsoluteError()],
)

In [70]:
model(i[0])

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[ 0.02869609],
       [-0.25832194],
       [-0.20421773],
       [-0.8842004 ],
       [-1.383323  ],
       [-0.5182167 ],
       [-0.06878534],
       [-0.7596993 ],
       [-0.93806046],
       [ 0.17720091],
       [ 0.14754361],
       [ 0.05606666],
       [-0.0100556 ],
       [-0.82166153],
       [-0.4795907 ],
       [ 0.11973408],
       [-0.6832311 ],
       [-0.6785938 ],
       [-0.2095502 ],
       [-0.90675706],
       [-0.88212216],
       [-0.41766584],
       [-0.2117496 ],
       [-0.12375206],
       [-0.6620742 ],
       [-0.53762376],
       [-0.682418  ],
       [ 0.25978512],
       [-0.15521216],
       [-0.34864938],
       [-0.15839148],
       [-0.34697783]], dtype=float32)>

In [71]:
model.fit(
    train_feature_dataset,
    batch_size=32
)

14770/14770 ━━━━━━━━━━━━━━━━━━━━ 1584s 107ms/step - loss: 0.2719 - mean_absolute_error: 0.3811
